In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import itertools
import sys

sys.path.append("..")

from utility_functions import (calculate_model_performance,
                               plot_ROC,
                               one_hot_encode,
                               split_data_as,
                               grid_search_stratified,
                               shuffle)


def get_shapes(any_):
    for array in any_:
        try:
            print(array.shape)
        except:
            print("NONE")
    print("\n")


# ============= ACTIVATION FUNCTIONS ===============#

def sigmoid(Z, prime=False):
    # np.
    if prime:
        return sigmoid(Z) * (1 - sigmoid(Z))
    return 1 / (1 + np.exp(-Z))


def linear(Z, prime=False):
    if prime:
        return np.ones_like(Z)
    return Z


def relu(Z, alpha=0.01, prime=False):
    if prime:
        Z_relu = np.ones_like(Z, dtype=np.float64)
        Z_relu[Z < 0] = alpha
        return Z_relu
    return np.where(Z < 0, alpha * Z, Z)


def tanh(Z, prime=False):
    # np.tanh() could be used directly to speed this up
    if prime:
        return 1 - np.power(tanh(Z), 2)
    return (2 / (1 + np.exp(-2 * Z))) - 1


def elu(Z, prime=False):
    # https://mlfromscratch.com/activation-functions-explained/#/
    alpha = 0.2
    if prime:
        return np.where(Z < 0, alpha * (np.exp(Z)), 1)
    return np.where(Z < 0, alpha * (np.exp(Z) - 1), Z)


def softmax(Z, prime=False):
    # https://deepnotes.io/softmax-crossentropy
    # max(Z) term is added to stabilise the function.
    exps = np.exp(Z - np.max(Z))
    return exps / np.sum(exps, axis=0)


# ============== LOSS FUNCTIONS ===============#

# https://deepnotes.io/softmax-crossentropy
EPSILON = 1e-8


def calculate_error(Y, Y_hat):
    # Y and Y_hat should be in the form of (no_of_classes, no_of_training_examples)
    m = Y.shape[1]
    return -np.sum(Y * np.log(Y_hat + EPSILON)) / m


# References
# https://mc.ai/multilayered-neural-network-from-scratch-using-python/
# https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/
# https://www.coursera.org/learn/machine-learning/home/week/5
# https://www.coursera.org/specializations/deep-learning
# https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network.py
# https://github.com/JWarmenhoven/Coursera-Machine-Learning

class NeuralNetwork:

    def __init__(
            self,
            input_layer: tuple,
            hidden_layer: list,  # list of tuples
            output_layer: int,
            batch_size=16,
            alpha=1,
            epoch=500,
            random_state=42,
            verbose=True,
            metrics="accuracy"
    ):
        self.input_layer = input_layer
        self.hidden_layer = hidden_layer
        self.output_layer = output_layer
        self.mini_batch_size = batch_size
        self.alpha = alpha
        self.epoch = epoch
        self.seed = random_state
        self.verbose = verbose
        self.metrics = metrics

        self.layers = len(self.weight_set_dimensions) + 1


    def __str__(self):
        parameters = (
            "Input layer: {0}\n"
            "Hidden layer: {1}\n"
            "Output layer: {2}\n"
            "Batch size: {3}\n"
            "Learning rate: {4}\n"
            "Epoch: {5}\n"
            "Seed: {6}\n"
            "Verbose: {7}\n"
            "Metric: {8}"
        ).format(
            self.input_layer,
            " - ".join(map(str, self.hidden_layer)),
            self.output_layer,
            self.mini_batch_size,
            self.alpha,
            self.epoch,
            self.seed,
            self.verbose,
            self.metrics
        )
        return parameters

    def get_A(self, X):
        A, _ = self.forwardpass(X)
        return A

    def get_Z(self, X):
        _, Z = self.forwardpass(X)
        return Z

    def display_information(self, X, Y, epoch_no):
        model_performance_metrics = calculate_model_performance(
            np.argmax(Y, axis=0),
            self.predict(X)
        )
        print("%s: %.10f - epoch %s    iteration %s - loss %.20f" % (
            self.metrics,
            model_performance_metrics[self.metrics],
            epoch_no,
            self.no_of_iterations,
            calculate_error(Y,
                            self.get_A(X)[-1])
        )
              )

    def get_dimensions_and_activations(self):
        self.dimensions = []
        self.activation_functions = []

        self.dimensions.append(self.input_layer[0])
        self.activation_functions.append(self.input_layer[1])

        for dim, act_func in self.hidden_layer:
            self.dimensions.append(dim)
            self.activation_functions.append(act_func)

        self.dimensions.append(self.output_layer)

    @property
    def weight_set_dimensions(self):
        self.get_dimensions_and_activations()
        a, b = itertools.tee(self.dimensions[::-1])
        next(b, None)
        weight_set_dimensions = list(zip(a, b))[::-1]
        return weight_set_dimensions

    def initialise_weights(self, layer=None):
        self.W = np.empty_like(range(self.layers), dtype=object)
        self.B = np.empty_like(range(self.layers), dtype=object)
        self.W[0] = None
        self.B[0] = None
        for layer, (y, x) in zip(range(1, self.layers), self.weight_set_dimensions):
            np.random.seed(self.seed)
            self.W[layer] = np.random.rand(y, x) / np.sqrt(self.dimensions[layer - 1])
            self.B[layer] = np.random.rand(y, 1)

    def forwardpass(self, X):
        Z = np.empty_like(range(self.layers), dtype=object)
        A = np.empty_like(range(self.layers), dtype=object)
        A[0] = X
        Z[0] = None
        for layer in range(1, self.layers):
            # activation_function starts from 0 whereas layer starts from 1
            active_function = self.activation_functions[layer - 1]
            arg_to_pass_to_eval = "(Z[layer])"

            Z[layer] = self.W[layer] @ A[layer - 1] + self.B[layer]
            A[layer] = eval(active_function + arg_to_pass_to_eval)
        return A, Z

    def backpropagation(self, Y, A, Z):
        self.delta = np.empty_like(range(self.layers), dtype=object)
        self.delta[0] = None

        self.gradient_W = np.empty_like(range(self.layers), dtype=object)
        self.gradient_B = np.empty_like(range(self.layers), dtype=object)
        self.gradient_W[0] = None
        self.gradient_B[0] = None

        self.delta[-1] = A[-1] - Y

        # We substract 1 here as delta_final is calculated seperately above
        for layer in reversed(range(1, self.layers - 1)):
            # 1 is substracted from layer as activation_functions start indexing from 0
            active_function = self.activation_functions[layer - 1]
            arg_to_pass_to_eval = "(Z[layer], prime=True)"

            self.delta[layer] = (
                    self.W[layer + 1].T @ self.delta[layer + 1] *
                    eval(active_function + arg_to_pass_to_eval)
            )

            # calculate the gradient

        for layer in range(1, self.layers):
            self.gradient_W[layer] = (self.delta[layer] @ A[layer - 1].T) / self.m
            self.gradient_B[layer] = np.sum(self.delta[layer], axis=1, keepdims=True) / self.m

        # update the weights
        for layer in range(1, self.layers):
            self.W[layer] -= self.alpha * self.gradient_W[layer]
            self.B[layer] -= self.alpha * self.gradient_B[layer]

    def fit(self, X, Y):
        self.m = X.shape[1] # where (no_of_features, no_of_training_examples)
        self.initialise_weights()

        # By default the method is SGD(Stochastic Gradient Descent) if one wishes to use
        # the whole batch, simply pass the number of traning examples available as the
        # batch size when instantiating the class
        self.no_of_iterations = 0
        shuffled = np.arange(self.m)
        if self.verbose:
            print("Initialising weights...")
            print("Starting the training...")
            print("Initial cost: %.10f\n" % calculate_error(Y, self.get_A(X)[-1]))
        for epoch_no in range(1, self.epoch + 1):
            np.random.shuffle(shuffled)
            X_shuffled = X[:, shuffled]
            Y_shuffled = Y[:, shuffled]
            for i in range(0, self.m, self.mini_batch_size):
                self.no_of_iterations += 1
                X_mini_batch = X_shuffled[:, i: i + self.mini_batch_size]
                Y_mini_batch = Y_shuffled[:, i: i + self.mini_batch_size]

                A, Z = self.forwardpass(X_mini_batch)
                self.backpropagation(Y_mini_batch, A, Z)
                if self.no_of_iterations % 5000 == 0 and self.verbose:
                    self.display_information(X, Y, epoch_no)

    def predict(
            self,
            X: np.ndarray,
            return_prob_matrix=False
    ):
        """Predict the output given the training data.

            Returns the predicted values in two forms:

            1.either by picking up the highest value along the columns for every row,
                i.e. "np.argmax(self.A[-1].T, axis=1)"
            2.or by returning a matrix that is in the shape of Y.T where each column
                represents the probability of the instance belonging to that class.
                Please note that every column in Y.T represents a class. To be able to
                return the probability matrix, the final activation function must be
                softmax!
                i.e. "array([0.9650488423, 0.0354737543, 0.0005225966])"

        Args:
            X (numpy.ndarray): Training set in the shape of
                (no_of_features, no_of_training examples).
            return_prob_matrix (bool, optional): Returns the probability matrix if True.
                Defaults to False.

        Returns:
            numpy.ndarray:

            if return_prob_matrix is False, the output is in the shape of
                (no_of_training_examples, 1)
            if return_prob_matrix is True, the output is in the shape of
                (no_of_training_examples, no_of_features)
        """
        A, Z = self.forwardpass(X)
        if return_prob_matrix:
            np.set_printoptions(precision=10, suppress=True)
            return A[-1].T
        return np.argmax(A[-1].T, axis=1)

# Testing with benchmark datasets

## 1.Iris Dataset

In [2]:
from sklearn.datasets import load_iris
data = load_iris()

x = data.data[:,[0,2]]
y = data.target

X = x.T
Y = one_hot_encode(y).T

# test, train = split_data_as(x, y, train=0.8, test=0.2)

# X_train = train[:, :-1]
# Y_train = train[:, -1].reshape(-1, 1)

In [3]:
print(X.shape)
print(Y.shape)
# print("\n")
# print(X_test.shape)
# print(Y_test.shape)
# print("\n")
# print(X_validation.shape)
# print(Y_validation.shape)

(2, 150)
(3, 150)


In [31]:
x = np.array(
    [
        [1,2,3],
        [4,5,6],
        [7,8,9],
        [11,12,13],
        [14,15,16],
        [17,18,19],
        [20,21,22],
        [23,24,25],
        [26,27,28],
        [29,30,31],
        [32,33,34],
        [35,36,37]
    ]
)

y = np.array(
    [
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [2],
        [2],
        [2],
        [2]
    ]
)

In [32]:
def shuffle(X, y):
    dataset = np.c_[X,y]
    shuffled = np.arange(len(dataset))
    np.random.shuffle(shuffled)
    return dataset[shuffled]

In [38]:
n_fold=3

In [39]:
dataset_shuffled = np.c_[x,y]
classes, counts = np.unique(y, return_counts=True)
classes = np.int_(classes)
counts_perc = counts / y.shape[0]

classes_holder_dict = {}
for class_ in classes:
    classes_holder_dict[class_] = dataset_shuffled[dataset_shuffled[:, -1] == class_]

fold_size = y.shape[0] / n_fold

train_dataset = {}
array_training = np.empty_like(classes, dtype=object)
test_dataset = {}
array_test = np.empty_like(classes, dtype=object)
for fold in range(n_fold):
    for class_ in classes:
        grab = np.int(fold_size * counts_perc[class_])
        array_test[class_] = classes_holder_dict[class_][fold * grab:(fold + 1) * grab ,:]
        array_training[class_] = np.delete(
            classes_holder_dict[class_],
            (range(fold * grab, (fold + 1) * grab)),
            axis=0
        )

    train_dataset[fold] = np.concatenate(array_training)
    test_dataset[fold] = np.concatenate(array_test)

In [40]:
dataset_shuffled

array([[ 1,  2,  3,  1],
       [ 4,  5,  6,  1],
       [ 7,  8,  9,  1],
       [11, 12, 13,  1],
       [14, 15, 16,  0],
       [17, 18, 19,  0],
       [20, 21, 22,  0],
       [23, 24, 25,  0],
       [26, 27, 28,  2],
       [29, 30, 31,  2],
       [32, 33, 34,  2],
       [35, 36, 37,  2]])

In [50]:
train_dataset[2]

array([[14, 15, 16,  0],
       [17, 18, 19,  0],
       [23, 24, 25,  0],
       [ 1,  2,  3,  1],
       [ 4,  5,  6,  1],
       [11, 12, 13,  1],
       [26, 27, 28,  2],
       [29, 30, 31,  2],
       [35, 36, 37,  2]])

In [48]:
test_dataset[2]

array([[20, 21, 22,  0],
       [ 7,  8,  9,  1],
       [32, 33, 34,  2]])

In [25]:
for fold in train_dataset:
    for index, row in enumerate(train_dataset[fold]):
        print(index, row)
    print("NEXT--")

0 [4.9 1.4 0. ]
1 [5.  1.2 0. ]
2 [4.6 1.4 0. ]
3 [5.1 1.5 0. ]
4 [5.  1.6 0. ]
5 [5.  1.4 0. ]
6 [4.6 1.5 0. ]
7 [5.  1.6 0. ]
8 [5.7 1.7 0. ]
9 [4.6 1.  0. ]
10 [4.9 1.4 0. ]
11 [5.  1.4 0. ]
12 [5.1 1.9 0. ]
13 [4.6 1.4 0. ]
14 [5.4 1.3 0. ]
15 [5.1 1.7 0. ]
16 [5.2 1.4 0. ]
17 [5.4 1.7 0. ]
18 [5.3 1.5 0. ]
19 [5.1 1.5 0. ]
20 [5.5 1.4 0. ]
21 [4.8 1.9 0. ]
22 [5.  1.3 0. ]
23 [4.3 1.1 0. ]
24 [4.8 1.4 0. ]
25 [5.6 4.2 1. ]
26 [6.2 4.5 1. ]
27 [5.5 3.7 1. ]
28 [5.7 4.2 1. ]
29 [5.5 4.4 1. ]
30 [6.5 4.6 1. ]
31 [7.  4.7 1. ]
32 [6.7 4.4 1. ]
33 [5.6 4.5 1. ]
34 [5.7 4.5 1. ]
35 [5.4 4.5 1. ]
36 [6.7 4.7 1. ]
37 [5.5 4.  1. ]
38 [5.9 4.8 1. ]
39 [5.7 4.1 1. ]
40 [5.9 4.2 1. ]
41 [6.  5.1 1. ]
42 [5.7 3.5 1. ]
43 [6.3 4.9 1. ]
44 [4.9 3.3 1. ]
45 [6.  4.5 1. ]
46 [6.7 5.  1. ]
47 [5.5 4.  1. ]
48 [5.2 3.9 1. ]
49 [5.8 4.1 1. ]
50 [6.8 5.5 2. ]
51 [4.9 4.5 2. ]
52 [6.5 5.8 2. ]
53 [6.3 5.6 2. ]
54 [7.7 6.7 2. ]
55 [6.9 5.1 2. ]
56 [6.5 5.5 2. ]
57 [7.7 6.9 2. ]
58 [6.7 5.6 2. ]
59 [6.9

In [26]:
x

array([[5.1, 1.4],
       [4.9, 1.4],
       [4.7, 1.3],
       [4.6, 1.5],
       [5. , 1.4],
       [5.4, 1.7],
       [4.6, 1.4],
       [5. , 1.5],
       [4.4, 1.4],
       [4.9, 1.5],
       [5.4, 1.5],
       [4.8, 1.6],
       [4.8, 1.4],
       [4.3, 1.1],
       [5.8, 1.2],
       [5.7, 1.5],
       [5.4, 1.3],
       [5.1, 1.4],
       [5.7, 1.7],
       [5.1, 1.5],
       [5.4, 1.7],
       [5.1, 1.5],
       [4.6, 1. ],
       [5.1, 1.7],
       [4.8, 1.9],
       [5. , 1.6],
       [5. , 1.6],
       [5.2, 1.5],
       [5.2, 1.4],
       [4.7, 1.6],
       [4.8, 1.6],
       [5.4, 1.5],
       [5.2, 1.5],
       [5.5, 1.4],
       [4.9, 1.5],
       [5. , 1.2],
       [5.5, 1.3],
       [4.9, 1.4],
       [4.4, 1.3],
       [5.1, 1.5],
       [5. , 1.3],
       [4.5, 1.3],
       [4.4, 1.3],
       [5. , 1.6],
       [5.1, 1.9],
       [4.8, 1.4],
       [5.1, 1.6],
       [4.6, 1.4],
       [5.3, 1.5],
       [5. , 1.4],
       [7. , 4.7],
       [6.4, 4.5],
       [6.9,

In [12]:
train_dataset[1]

array([[5.2, 1.5, 0. ],
       [5.4, 1.5, 0. ],
       [5.3, 1.5, 0. ],
       [5.8, 1.2, 0. ],
       [4.8, 1.4, 0. ],
       [4.4, 1.3, 0. ],
       [4.5, 1.3, 0. ],
       [5.5, 1.3, 0. ],
       [5.4, 1.5, 0. ],
       [5.1, 1.5, 0. ],
       [5. , 1.4, 0. ],
       [4.7, 1.3, 0. ],
       [5.5, 1.4, 0. ],
       [5.4, 1.3, 0. ],
       [4.6, 1. , 0. ],
       [5. , 1.5, 0. ],
       [4.9, 1.4, 0. ],
       [4.3, 1.1, 0. ],
       [4.9, 1.5, 0. ],
       [4.6, 1.5, 0. ],
       [5. , 1.3, 0. ],
       [4.8, 1.6, 0. ],
       [5.4, 1.7, 0. ],
       [5.1, 1.6, 0. ],
       [5.1, 1.5, 0. ],
       [5. , 1.6, 0. ],
       [5.1, 1.5, 0. ],
       [5.1, 1.4, 0. ],
       [5. , 1.2, 0. ],
       [4.7, 1.6, 0. ],
       [4.8, 1.9, 0. ],
       [5.4, 1.7, 0. ],
       [4.6, 1.4, 0. ],
       [4.8, 1.6, 0. ],
       [5.7, 1.5, 0. ],
       [5.2, 1.4, 0. ],
       [4.4, 1.3, 0. ],
       [4.4, 1.4, 0. ],
       [4.9, 1.5, 0. ],
       [5.2, 1.5, 0. ],
       [7. , 4.7, 1. ],
       [5.8, 4.1

In [ ]:
results_dict_all_models, results_average_dict, models = grid_search_stratified(
    x,
    y,
    clf=NeuralNetwork,
    metrics=["F1", "accuracy"],
    sort_by = "accuracy",
    n_fold=10,
    param_grid_dict={
        'batch_size': [8, 16, 32],
        'input_layer': [(2, 'relu'), (2, 'tanh')],
        'hidden_layer': [
            [(4,'relu'), (4,'relu'), (4,'softmax')],
            [(4,'sigmoid'),(4,'softmax')]
        ],
        'output_layer': [3],
        'alpha': [1, 2, 4],
        'verbose': [False],
        'epoch': [100]
    }
)

In [ ]:
results_average_dict

In [ ]:
results_dict_all_models["model_17"]

In [ ]:
print(models["model_1"])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
dt = data.data[:,[0,2]]
x_min, x_max = dt[:, 0].min() - 1, dt[:, 0].max() + 1
y_min, y_max = dt[:, 1].min() - 1, dt[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# here "model" is your model's prediction (classification) function
Z = models["model_1"].predict(np.c_[xx.ravel(), yy.ravel()].T) 

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx, yy, Z,alpha=0.4)
#plt.axis('off')

plt.scatter(dt[:, 0], dt[:, 1], c=y,s=20, edgecolor='k')
plt.xlabel('sepal length')
plt.ylabel('petal length')

## 2.Make Moons dataset

In [ ]:
from sklearn.datasets import make_moons

x,y =make_moons(n_samples=1500, noise=.05)
X = x.T
Y = one_hot_encode(y).T

print(X.shape)
print(Y.shape)

In [ ]:
model = NeuralNetwork(
    input_layer=(2, "sigmoid"),
    hidden_layer=[(8, "tanh"),
                  (6, "relu"),
                  (4, "softmax")],
    output_layer=2,
    batch_size=64,
    alpha=0.5,
    epoch=2500,
    random_state=42
)


model.fit(X, Y)

In [ ]:
### Decision Boundaries
%matplotlib inline
import matplotlib.pyplot as plt
dt = x
x_min, x_max = dt[:, 0].min() - 0.5, dt[:, 0].max() + 0.5
y_min, y_max = dt[:, 1].min() - 0.5, dt[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

# here "model" is your model's prediction (classification) function
Z = model.predict(np.c_[xx.ravel(), yy.ravel()].T) 

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx, yy, Z,alpha=0.4)
#plt.axis('off')
plt.scatter(dt[:, 0], dt[:, 1], c=y, s=20, edgecolor='k')
plt.title('Decision Boundaries')

## 3.Andrew NG Assignment 2 Dataset

In [ ]:
ex2data2 = np.loadtxt("../ex2/data/ex2data2.txt", delimiter=",")

x = ex2data2[:, :-1]
y = ex2data2[:, -1]

X = x.T
Y = one_hot_encode(y).T

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
model = NeuralNetwork(
    input_layer=(X.shape[0], "relu"),
    hidden_layer=[(4, "relu"),
                  (4, "softmax")],
    output_layer=Y.shape[0],
    batch_size=16,
    alpha=4,
    epoch=5000,
    random_state=42
)

model.fit(X,Y)

In [ ]:
results_dict_all_models, results_average_dict, models = grid_search_stratified(
    x,
    y,
    clf=NeuralNetwork,
    metrics=["F1", "accuracy"],
    sort_by = "accuracy",
    n_fold=6,
    param_grid_dict={
        'batch_size': [16, 32],
        'input_layer': [(2, 'relu')],
        'hidden_layer': [
            [(4,'relu'), (4,'relu'), (4,'softmax')],
            [(4,'sigmoid'),(4,'softmax')]
        ],
        'output_layer': [2],
        'alpha': [2, 4],
        'verbose': [False],
        'epoch': [5000]
    }
)

In [ ]:
results_average_dict

In [ ]:
print(models["model_5"])

In [ ]:
### Decision Boundaries
%matplotlib inline
import matplotlib.pyplot as plt
X = ex2data2

x1_min, x1_max = X[:, 0].min() - 0.3, X[:, 0].max() + 0.3,
x2_min, x2_max = X[:, 1].min() - 0.3, X[:, 1].max() + 0.3,
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max), np.linspace(x2_min, x2_max))

# here "model" is your model's prediction (classification) function
Z = models["model_5"].predict(np.c_[xx1.ravel(), xx2.ravel()].T) 

negatives = ex2data2[ex2data2[:, -1] == 0]
positives = ex2data2[ex2data2[:, -1] == 1]

# Put the result into a color plot
Z = Z.reshape(xx1.shape)
plt.figure(figsize=(12,8))
plt.contourf(xx1, xx2, Z,alpha=0.4)
#plt.axis('off')
plt.scatter(negatives[:, 0], negatives[:, 1],s=50, color='k')
plt.scatter(positives[:, 0], positives[:, 1],s=50, color='r')
plt.title('Decision Boundaries')

plt.contour(xx1, xx2, Z, [0.5], linewidths=2, colors="g")

# MNIST dataset

In [ ]:
from scipy.io import loadmat
data = loadmat('../ex3/data/ex3data1.mat')
data.keys()

In [ ]:
x = data["X"]
y = data["y"]
y[y==10] = 0

In [ ]:
X = x.T
Y = one_hot_encode(y).T

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20,20))

sample = np.random.choice(data["X"].shape[0], 20)
ax.imshow(data["X"][sample,1:].reshape(-1,20).T)
ax.axis('off');

In [ ]:
results_dict_all_models, results_average_dict, models = grid_search_stratified(
    x,
    y,
    clf=NeuralNetwork,
    metrics=["F1", "accuracy"],
    sort_by = "F1",
    n_fold=5,
    param_grid_dict={
        'batch_size': [16, 32, 64],
        'input_layer': [(x.shape[1], 'relu')],
        'hidden_layer': [
            [(50,'relu'), (10,'softmax')],
            [(25,'relu'), (10,'sigmoid'), (10,'softmax')],
            [(25,'relu'), (10,'relu'), (10,'softmax')]
        ],
        'output_layer': [10],
        'alpha': [3,4,6],
        'verbose': [False],
        'epoch': [500]
    }
)

In [ ]:
results_average_dict

In [ ]:
results_dict_all_models["model_21"]

In [ ]:
print(models["model_21"])

In [ ]:
for index, (act, predicted) in enumerate(zip(np.argmax(Y,axis=0), models["model_21"].predict(X))):
    if act != predicted:
        fig, ax = plt.subplots(figsize = (2,2))
        ax.set_title("%s: act %s --- predicted %s" %(index, act, predicted))
        ax.imshow(X[:, index].reshape(-1,20).T)
        ax.axis('off');